# Soil Moisture Drydowns

The following script loads a time series of root-zone soil moisture obtained from the Kansas Mesonet, extracts drydown periods, and characterizes each drydown period by using the exponential decay rate fitted to each drydown period.

A drydown represents a soil moisture timeseries of consecutive days without rainfall events. Formally, any particular day is part of a drydown period if $SWC_{t} < SWC_{t-1}$ In this particular script we will consider small positive changes in soil moisture between consecutive days as part of the drydown. This small positive changes could be due to small rainfall events, fog, dew, and even noise in the signal.

Let's first import some python modules, all of which are available with the Anaconda package:

In [38]:
# Import Python modules
import pandas as pd
import numpy as np
from pprint import pprint
from scipy.optimize import curve_fit
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

## Model description

$$ SWC = A e^{-\frac{t}{\tau}} + \theta_{res}$$

SWC = Soil water content in $m^{3}/m^{3}$

A = The initial soil water content $m^{3}/m^{3}$. Soil water at time t=0

t = Days since rainfall event

$\tau$ = Constant the modulates the rate at which the soil dries

$\theta_{res}$ = Residual soil water content $m^{3}/m^{3}$. This is a lower limit that needs to be determined from the data. Alternatively, this value can approximated using pedo-transfer functions (e.g. residual water content as a function of percent sand content)


## Intuition

Before we even dive into the script I want to show you what several examples of drydowns based on the exponential model that we will use later on in this script. Soil moisture can increase rapidly during and soon after rainfall events. In subsequent days soil moisture dries at a decreasing rate resembling an exponential decay, which depends on atmospheric demand, soil type and structure, and the presence of vegetation. The lines below represent possible decays under different scenarios.

> Using an exponentail decay function to model drydowns is an effective and simple empirical method. Drydowns resembling an exponential decay are typical of soil moisture timeries obtained with in-situ sensors. Applying this to remote sensing data should work, but drydowns may not always exhibit a smooth exponential decay. Despite of its simplicity, this simple empirical model provides a quantitative framework for comparing drydowns from different times of the year and locations.

In [39]:
# Define model using an anonymous lamda function
A = 0.15 # Initial soil moisture minus the residual moisture. 

# In this case a sensor in the ground would have reported a value of 0.25 (0.15 + 0.10)
theta_res = 0.10 # m3/m3 A common value for silt loam soils.
model = lambda t,tau: A * np.exp(-t/tau) + theta_res;
xrange = np.arange(30)

# Create figure
f = figure(width=400, height=300)

# Rapid decay. Typical of summer days with actively growing vegetation
f.line(xrange, model(xrange,5), line_color='black')     
f.line(xrange, model(np.arange(30),10), line_color='purple')

# Intermediate drydowns typical of spring and fall, or perhaps fine-texture soils
f.line(xrange,model(xrange,25), line_color='red')
f.line(xrange,model(xrange,50), line_color='tomato')

# Drydowns typical of winter, exhibiting low or almost no change in soil moisture
# This is due to low atmopsheric demand, abscense of vegetation, 
# partially frozen soils,
# or even low viscosity of water
f.line(xrange,model(xrange,100), line_color='blue')
f.line(xrange,model(xrange,500), line_color='green')

f.xaxis.axis_label = 'Days since last'
f.yaxis.axis_label = 'Volumetric Water Content m^3/m^3'
show(f)


## Load dataset

Test dataset was compile for the following spatial and temporal parameters:

    start_date = '1-jan-2016'

    end_date = '18-may-2018'

    lat = 38.730981

    lon = -97.416299

    missing values = NaN

Load data and convert dates in to Pandas datetime format and check that the conversion was successful

In [40]:
# Load data
df = pd.read_csv('../datasets/gypsum_ks_daily_2018.csv')
df.head()


,TIMESTAMP,STATION,PRESSUREAVG,PRESSUREMAX,PRESSUREMIN,SLPAVG,TEMP2MAVG,TEMP2MMIN,TEMP2MMAX,TEMP10MAVG,TEMP10MMIN,TEMP10MMAX,RELHUM2MAVG,RELHUM2MMAX,RELHUM2MMIN,RELHUM10MAVG,RELHUM10MMAX,RELHUM10MMIN,VPDEFAVG,PRECIP,SRAVG,SR,WSPD2MAVG,WSPD2MMAX,WSPD10MAVG,WSPD10MMAX,WDIR2M,WDIR2MSTD,WDIR10M,WDIR10MSTD,SOILTMP5AVG,SOILTMP5MAX,SOILTMP5MIN,SOILTMP10AVG,SOILTMP10MAX,SOILTMP10MIN,SOILTMP5AVG655,SOILTMP10AVG655,SOILTMP20AVG655,SOILTMP50AVG655,VWC5CM,VWC10CM,VWC20CM,VWC50CM
0,1/1/18 0:00,Gypsum,99.44,100.03,98.73,104.44,-15.15,-19.56,-11.00,-15.31,-19.56,-11.00,56.63,71.35,40.37,57.34,71.35,40.37,0.08,0.0,110.92,9.58,4.62,9.06,5.65,9.41,6.17,15.20,7.13,13.36,-2.99,-1.90,-4.17,-1.66,-1.18,-2.45,-1.33,-1.14,0.74,3.50,0.1377,0.1167,0.2665,0.2203
1,1/2/18 0:00,Gypsum,99.79,100.14,99.40,104.88,-16.48,-22.10,-10.40,-16.38,-22.10,-10.40,57.44,76.06,35.55,57.08,76.06,35.55,0.08,0.0,126.78,10.95,1.42,4.41,1.95,5.19,26.68,50.50,30.52,48.24,-3.74,-2.08,-5.20,-2.52,-1.56,-3.46,-2.10,-1.82,0.28,3.13,0.1234,0.1021,0.2642,0.2196
2,1/3/18 0:00,Gypsum,98.87,99.52,97.94,103.81,-11.03,-20.64,-2.71,-10.66,-20.64,-2.71,50.74,80.24,26.40,49.24,80.24,26.40,0.16,0.0,118.89,10.27,2.48,8.27,3.06,9.17,210.40,31.50,201.24,42.86,-3.41,-1.72,-5.13,-2.51,-1.49,-3.61,-2.21,-1.93,-0.08,2.76,0.1206,0.0965,0.2353,0.2189
3,1/4/18 0:00,Gypsum,98.22,98.54,97.90,102.99,-5.83,-11.79,0.24,-5.01,-11.79,0.24,60.17,81.65,42.80,58.91,81.65,42.80,0.17,0.0,95.84,8.28,2.43,8.55,3.04,9.39,296.14,66.11,320.88,69.61,-2.50,-0.95,-3.65,-1.89,-0.98,-2.67,-1.60,-1.46,-0.21,2.45,0.1235,0.0973,0.2094,0.2182
4,1/5/18 0:00,Gypsum,98.10,98.42,97.75,102.88,-4.73,-14.22,5.36,-4.23,-14.22,5.36,58.75,85.26,30.29,57.83,85.26,30.29,0.23,0.0,126.05,10.89,1.97,7.43,2.59,8.37,178.90,36.47,178.09,34.39,-2.17,-0.46,-3.91,-1.71,-0.72,-2.81,-1.54,-1.38,-0.25,2.25,0.1249,0.0976,0.2047,0.2180


In [41]:
# Convert date strings into pandas datetie format
df['TIMESTAMP'] = pd.to_datetime(data["TIMESTAMP"], format='%m/%d/%y %H:%M')
df.insert(1,'DOY',df['TIMESTAMP'].dt.dayofyear)
df[['TIMESTAMP','DOY']].head()

,TIMESTAMP,DOY
0,2018-01-01,1
1,2018-01-02,2
2,2018-01-03,3
3,2018-01-04,4
4,2018-01-05,5


In [44]:
# Plot timeseries of soil moisture and EDDI
f = figure(width=600, height=300, x_axis_type='datetime')
f.line(df['TIMESTAMP'], df['VWC5CM'])
f.yaxis.axis_label = 'Volumetric Water Content m³/m³'
show(f)

The SMAP signal shows some increases and decreases that might not be real. Particularly the dramatic reductions in soil moisture occurrying in the span of one or two days (see sharp reduction of soil moisture change near March-2016, this will also be evident in the plot below when we overlay the drydown periods)

In [82]:
# Find residual volumetric water content
# We can also approximate this value by taking the first or fifth percentiles for example
theta_res = df['VWC5CM'].min()
print(theta_res)

0.1206


In [83]:
# Iterate over soil moisture timeseries to retrieve drydowns
drydown_counter = 0
day_counter = 0

# Initialize drydowns
drydowns = [{'date':[],'VWC':[],'doy':[],'days':[],'length':[]}]

# We start the loop on the second day
for i in range(1,len(df)):
    delta = df["VWC5CM"][i] - data["VWC5CM"][i-1]
    
    if delta < 0:
        drydowns[drydown_counter]['date'].append(df['TIMESTAMP'][i])
        drydowns[drydown_counter]['VWC'].append(df['VWC5CM'][i])
        drydowns[drydown_counter]['doy'].append(df['DOY'][i])
        drydowns[drydown_counter]['days'].append(day_counter)
        drydowns[drydown_counter]['length'] = day_counter+1
        day_counter += 1
        
    else:
        drydown_counter += 1
        day_counter = 0
        drydowns.append({'date':[],'VWC':[],'doy':[],'days':[],'length':[]})


In [84]:
drydowns_clean = []
for i in range(len(drydowns)):
    if (drydowns[i]['length'] != [] and drydowns[i]['length'] > 5):
        drydowns_clean.append(drydowns[i])
        
print('There are a total of',len(drydowns_clean),'drydowns')   


There are a total of 15 drydowns


In [85]:
# Print dictionary with all the data for the first drydown period.
# The pretty print module just makes the dictionary easy to read. Keys are sorted alphabetically
pprint(drydowns_clean[0])

{'VWC': [0.2929, 0.2837, 0.2743, 0.266, 0.2562, 0.2488],
 'date': [Timestamp('2018-04-28 00:00:00'),
          Timestamp('2018-04-29 00:00:00'),
          Timestamp('2018-04-30 00:00:00'),
          Timestamp('2018-05-01 00:00:00'),
          Timestamp('2018-05-02 00:00:00'),
          Timestamp('2018-05-03 00:00:00')],
 'days': [0, 1, 2, 3, 4, 5],
 'doy': [118, 119, 120, 121, 122, 123],
 'length': 6}


## Overlay soil moisture timeseries and extracted drydowns

In [86]:
f = figure(width=500, height=300, x_axis_type='datetime')
f.line(df['TIMESTAMP'], df['VWC5CM'])

for i in range(len(drydowns_clean)):
    f.line(drydowns_clean[i]['date'],drydowns_clean[i]['VWC'], line_color='red', line_width=2)
    
f.yaxis.axis_label = 'Volumetric Water Content m³/m³'
show(f)

## Overlay soil moisture timeseries, extracted drydowns, and fitted model

It's important to highlight that:

- the "x" variable, in this case $t$ needs to be defined first in the lambda function

- the lambda function needs to be defined in each iteration to ensure that $A$ is updated with the initial soil moisture of the current drydown period in the iteration process.

- It is possible to also fit $A$, but since we do know the value of the parameter from the soil moisture signal it is better to force the model thourhg this value and only optimize tau.

- The $\theta_{res} is considered constant. In other words, for any day of the year it assumed that the soil moisture tends towards this point.

- Values of $\tau$ are inversely related to the drydown rate

In [87]:
for i in range(len(drydowns_clean)):
    xdata = drydowns_clean[i]['days']
    ydata = drydowns_clean[i]['VWC']
    A = drydowns_clean[i]['VWC'][0] - theta_res # Initial soil moisture minus theta_res
    model = lambda t,tau: A * np.exp(-t/tau) + theta_res; # Define lambda function in each iteration
    par_opt, par_cov = curve_fit(model, xdata, ydata)
    drydowns_clean[i]['tau'] = par_opt[0]


In [97]:
f = figure(width=700,height=300, x_axis_type='datetime')
f.line(df['TIMESTAMP'], df['VWC5CM'])

for i in range(len(drydowns_clean)):
    #f.line(drydowns_clean[i]['date'],drydowns_clean[i]['VWC'], line_color='gray',line_width=4)
    A = drydowns_clean[i]['VWC'][0] - theta_res
    model = lambda t,tau: A * np.exp(-t/tau) + theta_res;
    f.line(drydowns_clean[i]['date'],model(np.array(drydowns_clean[i]['days']), drydowns_clean[i]['tau']),
           line_color='red',
           line_width=2)
    
show(f)
